In [1]:
import pandas as pd

# Data Merge

In [10]:
import json
with open('../SolarEdgeDataGathering/site_information.json', 'r') as file:
    site_information = json.load(file)
site_information

{'sites': {'count': 14,
  'site': [{'id': 103941,
    'name': 'Alan Knudson House',
    'accountId': 53921,
    'status': 'Active',
    'peakPower': 10.36,
    'lastUpdateTime': '2020-01-21',
    'currency': 'USD',
    'installationDate': '2015-01-31',
    'ptoDate': None,
    'notes': '',
    'type': 'Optimizers & Inverters',
    'location': {'country': 'United States',
     'state': 'Arizona',
     'city': 'Centennial Park',
     'address': 'Taylor Court 1745',
     'address2': '',
     'zip': '86021',
     'timeZone': 'America/Phoenix',
     'countryCode': 'US',
     'stateCode': 'AZ'},
    'alertQuantity': 0,
    'primaryModule': {'manufacturerName': 'RENESOLA',
     'modelName': 'SE11400A-US',
     'maximumPower': 310.0,
     'temperatureCoef': -0.4},
    'uris': {'PUBLIC_URL': 'https://monitoring.solaredge.com/solaredge-web/p/public?name=Alan Knudson Canopy',
     'DATA_PERIOD': '/site/103941/dataPeriod',
     'DETAILS': '/site/103941/details',
     'OVERVIEW': '/site/103941/over

In [9]:
import os

def read_production_data(house):
    path = f'../SolarEdgeDataGathering/data/{house}'
    dfs = []
#     production_df = pd.read_csv(f'data/{house}')
    for subdir, dir, files in os.walk(path):
        for file in files:
            file_path = os.path.join(subdir, file)
            df = pd.read_csv(file_path)
            dfs.append(df)
    production_df = pd.concat(dfs)
    return production_df

def read_weather_data(house):
    path = f'../SolarEdgeDataGathering/weather-data/{house}'
    weather_df = pd.read_csv(os.path.join(path, 'weather_data_hourly.csv'))
    return weather_df

def set_date_index(df, site):
    df_d = df.copy()
    df_d['date'] = pd.to_datetime(df['date'])
    df_d = df_d.set_index('date')
    df_d = df_d.tz_localize(site['location']['timeZone'], ambiguous='NaT', nonexistent='NaT')
    return df_d

def combine_prod_and_weather(site):
    # read production data
    production_df = read_production_data(site['id'])
    
    #set the date index
    production_df_t = set_date_index(production_df, site)
    
    # resmaple production data to hourly
    production_df_r = production_df_t.resample('1H').sum()
    
    # read in weather data
    weather_df = read_weather_data(site['id'])
    
    # set the date index
    weather_df_t = set_date_index(weather_df, site)
    
    # merge datasets
    combined_df = weather_df_t.merge(production_df_t, left_index=True, right_index=True, how='inner')
    combined_df = combined_df.rename(columns={'value': 'production'})
    
    site_id = site['id']
    folder = f'data/{site_id}'
    path = f'{folder}/production_weather_combination.csv'
    if not os.path.exists(path):
        os.mkdir(folder)
    combined_df.to_csv(path)

In [11]:
for site in site_information['sites']['site']:
    combine_prod_and_weather(site)

# Experiment
Keeping the experiment code just in case it becomes useful in the future. Won't run correctly right now

Read in both data frames to get an idea of what we need to combine. We'll start first with the production data

In [ ]:
import os

def read_production_data(house):
    path = f'data/{house}'
    dfs = []
#     production_df = pd.read_csv(f'data/{house}')
    for subdir, dir, files in os.walk(path):
        for file in files:
            file_path = os.path.join(subdir, file)
            df = pd.read_csv(file_path)
            dfs.append(df)
    production_df = pd.concat(dfs)
    return production_df

Need to construct a generic function that will combine the data for any site
For now, I'm going to store all of the combined data into multiple different sets
This data needs to be the final data that we are going to run our regression model on
This data also needs to be fully cleaned before we move it to regression
We will want to experiment with different data to see how the accuracy of the model improves. 
Some data sets might just include all sun hours. Some might just include the most relevant sun hours, whatever that means
We will also need to think about cross validation if we perform any, and how we want to split the data into train, validate, and test. I don't know that we'll need to perform cross validation on the simple model, so we just need to make sure that we get an even split into train and test for the first model run. 

Steps
1. Put all of the code into one function (or class) that will read in data from both sources, combine them together. We will save cleaning for later. Right now, we are going to store as much information as we feel like doing. 
2. Run all of the sites through that algorithm, and save the final file in as an actual CSV in the data folder. 
3. Start a new notebook, do some initial cleaning, and run through a simple linear regression model. 

In [ ]:
site = site_information['sites']['site'][0]
site

In [ ]:
production_df = read_production_data(site['name'])
production_df

Because the historical weather data only comes in 15 minute increments, we'll want to resample the production data to be every hour. First we'll want to set the index of our data frame to be the date. We'll have to set the 'date' column to be a datetime object

In [ ]:
production_df_t = production_df.copy()
production_df_t['date'] = pd.to_datetime(production_df['date'])
production_df_t = production_df_t.set_index('date')
production_df_t = production_df_t.tz_localize(site['location']['timeZone'])
production_df_t.head()

Now we can do the resampling

In [ ]:
production_df_r = production_df_t.resample('1H').sum()
production_df_r

Now we can read the historical weather data in for the same site

In [ ]:
def read_weather_data(house):
    path = f'weather_data/{house}'
    weather_df = pd.read_csv(os.path.join(path, 'weather_data_hourly.csv'))
    return weather_df

In [ ]:
weather_df = read_weather_data(site['name'])
weather_df

The time appears to be in the same timezone as the production data. For now, I am going to assume that that timezone is correct, and use it to merge the data. We'll want to convert the timezone and set it as the index as we did before

In [ ]:
weather_df_t = weather_df.copy()
weather_df_t['date'] = pd.to_datetime(weather_df['date'])
weather_df_t = weather_df_t.set_index('date')
weather_df_t = weather_df_t.tz_localize(site['location']['timeZone'])
weather_df_t.head()

we can merge the dataframes on their common index to get a combined df

In [ ]:
combined_df = weather_df_t.merge(production_df_t, left_index=True, right_index=True, how='inner')
combined_df = combined_df.rename(columns={'value': 'production'})
combined_df

In [ ]:
combined_df.to_csv("data/test_combination.csv")